In [1]:
tickers = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX',
           'VOX.MX','BBVA.MX','AC.MX','AMXL.MX',
           'BIMBOA.MX','GAPB.MX','C.MX',
           'AEROMEX.MX','WALMEX.MX','SAN.MX','GMEXICOB.MX' ]

In [2]:
# Se importan librerías 
import matplotlib
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.optimize as opt
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader.data as web

In [3]:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

In [4]:
def montecarlo(ret,rf,n_act):
    summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                                     'Vol':((252)**(.5))*ret.std()})
    Sigma = ret.cov()
    corr = ret.corr()
    rf = rf
    n_port = 100000
    n_act = n_act
    W = np.random.dirichlet(alpha=np.ones(n_act), size=n_port)
    W.shape
    W[:5]
    er = W.dot(summary["Media"])
    s = np.zeros(n_port)

    for i in range(n_port):
        s[i] = (W[i, :].T.dot(Sigma).dot(W[i, :]))**0.5
    rs = (er - rf) / s

    portafolios_montecarlo = pd.DataFrame(
        {
            "XLK.MX":  W[:, 0],
            "VOO.MX": W[:, 1],
            "VGT.MX": W[:, 2],
            'VHT.MX': W[:, 3],
            'VOX.MX': W[:, 4],
            'BBVA.MX': W[:, 5],
            'AC.MX': W[:, 6],
            'AMXL.MX': W[:, 7],
            'BIMBOA.MX': W[:, 8],
            'GAPB.MX': W[:, 9],
            'C.MX': W[:, 10],
            'AEROMEX.MX': W[:, 11],
            'WALMEX.MX': W[:, 12],
            'SAN.MX': W[:, 13],
            'GMEXICOB.MX': W[:, 14],
            "Media": er,
            "Vol": s,
            "RS": rs
        }
    )

    # Portafolio EMV (Eficinete en media varianza)
    emv = portafolios_montecarlo.loc[portafolios_montecarlo["RS"].idxmax()]
    # Portafolio mínima varianza
    minvar = portafolios_montecarlo.loc[portafolios_montecarlo["Vol"].idxmin()]
    
    DB = pd.DataFrame(columns={'EMV_MCarlo','minvar_Mcarlo'})
    DB['EMV_MCarlo']=emv
    DB['minvar_Mcarlo']=minvar
    

    return DB

    

In [5]:
def minvarMinimiza(ret,rf):
    summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                                         'Vol':((252)**(.5))*ret.std()})
    Sigma = ret.cov()
    corr = ret.corr()
    rf = rf
    S = np.diag(summary['Vol'])
    Eind = summary['Media'].values.astype(float)
    # Función objetivo
    def varianza(w, Sigma):
        return w.T.dot(Sigma).dot(w)
    # Número de activos
    n = len(Eind)
    # Dato inicial
    w0 = np.ones(n) / n
    # Cotas de las variables
    bnds = ((0, 1),) * n
    # Restricciones
    cons = {"type": "eq", "fun": lambda w: w.sum() - 1}
    # Portafolio de mínima varianza
    minvar = minimize(fun=varianza,
                      x0=w0,
                      args=(Sigma,),
                      bounds=bnds,
                      constraints=cons,
                      tol=1e-10)
    # Pesos, rendimiento, riesgo y razón de Sharpe del portafolio de mínima varianza
    w_minvar = minvar.x
    e_minvar = Eind.dot(w_minvar)
    s_minvar = (w_minvar.T.dot(Sigma).dot(w_minvar))**0.5
    rs_minvar = (e_minvar - rf) / s_minvar
    e_minvar, s_minvar, rs_minvar
    DB_b = pd.DataFrame(columns={'w_minvar'},index=tickers)
    DB_b['w_minvar']=w_minvar
    return DB_b

In [21]:
#EMV 
def EMV_Minimize(ret,rf,g):
    summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                                             'Vol':((252)**(.5))*ret.std()})
    rf=rf
    g = g
    corr = ret.corr()
    S = np.diag(summary['Vol'])
    Sigma = S.dot(corr).dot(S).astype(float)
    # 2. Eind: rendimientos esperados activos individuales
    Eind = summary['Media'].values.astype(float)
    # Función objetivo
    def menos_rs(w, Eind, rf, Sigma):
        ep = Eind.dot(w)
        sp = (w.T.dot(Sigma).dot(w))**0.5
        rs = (ep - rf) / sp
        return -rs
    # Número de activos
    n = len(Eind)
    # Dato inicial
    w0 = np.ones(n) / n
    # Cotas de las variables
    bnds = ((0, 1),) * n
    # Restricciones
    cons = {"type": "eq", "fun": lambda w: w.sum() - 1}
    emv = minimize(fun=menos_rs,
                   x0=w0,
                   args=(Eind, rf, Sigma),
                   bounds=bnds,
                   constraints=cons,
                   tol=1e-10)
    w_emv = emv.x
    e_emv = Eind.dot(w_emv)
    s_emv = (w_emv.T.dot(Sigma).dot(w_emv))**0.5
    rs_emv = (e_emv - rf) / s_emv
    w_opt = (e_emv - rf) / (g * s_emv**2)
    W = [w_emv[i]*w_opt for i in range(len(w_emv))]
    DB_c = pd.DataFrame(columns={'emv_opt'},index=tickers)
    DB_c['emv_opt']=W
    return DB_c

In [22]:
start_date = '2011-12-31'
end_date = '2016-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret= closes_prev.pct_change().dropna()
rf = 0.0546
g = 7

In [23]:
DB_Montecarlo = montecarlo(ret,rf,15)
DB_bminimiza = minvarMinimiza(ret,rf)
DB_C = EMV_Minimize(ret,rf,g)

In [24]:
DB_Montecarlo['minva_Minimize'] = DB_bminimiza['w_minvar']
DB_Montecarlo['emv_opt'] =DB_C['emv_opt']
Portafolios = DB_Montecarlo[:15]
Portafolios


,minvar_Mcarlo,EMV_MCarlo,minva_Minimize,emv_opt
XLK.MX,0.000791,0.045336,0.000000e+00,1.185212e-02
VOO.MX,0.006243,0.047828,7.846078e-05,1.266379e-02
VGT.MX,0.009334,0.036456,1.022655e-18,9.122720e-03
VHT.MX,0.000945,0.027147,4.798918e-05,7.094343e-03
VOX.MX,0.008856,0.030218,7.483720e-05,8.296460e-03
BBVA.MX,0.066294,0.129961,1.433381e-02,0.000000e+00
AC.MX,0.104193,0.033865,2.288242e-01,6.952350e-17
AMXL.MX,0.043233,0.045652,1.171255e-01,0.000000e+00
BIMBOA.MX,0.398635,0.128615,2.060264e-02,0.000000e+00
GAPB.MX,0.093736,0.005994,1.201675e-01,7.776826e-01


1.5294462282927155